In [ ]:
import numpy as np


class PoolModel:
    def __init__(
        self,
        admin_fee=0.5,
        rebalance_offset: float = 0.01,
        xcp_growth: float = 0.01,
        mode="linear",
        compound_mode="mul",
    ):
        self.vp = 1.0
        self.xcp_profit = 1.0
        self.xcp_profit_a = 1.0
        self.admin_claimed_fees = 0.0
        self.offset = rebalance_offset
        self.total_supply = 1.0
        self.xcp = 1.0
        self.xcp_growth = xcp_growth
        self.mode = mode
        self.admin_fee = admin_fee
        self.compound_mode = compound_mode
        # bookkeeping for plotting
        self.step_count = 0
        self.history = {
            "vp": [],
            "xcp_profit": [],
            "xcp_profit_a": [],
            "reserved_xcp_profit": [],
            "admin_claimed_fees": [],
            "xcp": [],
        }
        self.claim_marks = []  # list of (step, vp)
        self.rebalance_marks = []  # list of (step, vp)

    # ---------------- core actions ---------------- #

    def trade(self, rebalance_flag=False) -> None:
        xcp_new = self.xcp * (1.0 + self.xcp_growth)
        # xcp_new = self.xcp + self.xcp_growth
        self.xcp = xcp_new
        self.tweak_price(rebalance_flag)

    def claim_fees(self) -> float:
        if "linear" in self.mode:
            return self.claim_fees_linear()
        elif "sqrt" in self.mode:
            return self.claim_fees_sqrt()
        else:
            raise ValueError(f"Invalid mode: {self.mode}")

    def claim_fees_linear(self) -> float:
        delta = self.xcp_profit - self.xcp_profit_a
        if delta <= 0.0:
            return 0.0

        fee = delta / 2 * self.admin_fee

        frac = self.vp / (self.vp - fee) - 1
        admin_share = self.total_supply * frac

        vp = self.xcp / (self.total_supply + admin_share)
        if vp < 1.0:
            return 0.0
        self.vp = vp
        self.xcp = vp * self.total_supply
        if self.mode == "linear1":
            self.xcp_profit -= fee
        elif self.mode == "linear2":
            self.xcp_profit -= fee * 2
        self.xcp_profit_a = self.xcp_profit

        self.claim_marks.append((self.step_count, self.vp))
        return fee

    def claim_fees_sqrt(self) -> float:
        fee = self.admin_unclaimed_fees()

        frac = self.vp / (self.vp - fee) - 1
        admin_share = self.total_supply * frac

        vp = self.xcp / (self.total_supply + admin_share)
        if vp < 1.0:
            return 0.0

        if self.mode == "sqrt":
            self.xcp_profit = (
                np.sqrt(self.xcp_profit) * (1 - self.admin_fee)
                + np.sqrt(self.xcp_profit_a) * self.admin_fee
            ) ** 2
        elif self.mode == "sqrt_lin":
            self.xcp_profit = (
                self.xcp_profit * (1 - self.admin_fee) + self.xcp_profit_a * self.admin_fee
            )
        elif self.mode == "sqrt_admin":
            self.admin_claimed_fees += fee

        self.vp = vp
        self.xcp = vp * self.total_supply
        self.xcp_profit_a = self.xcp_profit

        self.claim_marks.append((self.step_count, self.vp))
        return fee

    def admin_unclaimed_fees(self) -> float:
        root_P = np.sqrt(self.xcp_profit)
        root_P_a = np.sqrt(self.xcp_profit_a)
        if self.mode == "sqrt_admin":
            admin_full_fees = (root_P - 1) * self.admin_fee
        else:
            admin_full_fees = (root_P - root_P_a) * self.admin_fee
        admin_unclaimed = admin_full_fees - self.admin_claimed_fees
        return max(0.0, admin_unclaimed)

    def tweak_price(self, rebalance_flag=False) -> float:
        vp_old = self.vp
        vp_new = self.xcp / self.total_supply
        if self.compound_mode == "add":
            # we must compound as vp_new/vp_old
            # but we can't compound whole body of xcp_profit, because we remove admin_claimed_fees
            # so we must compound only the part that is not admin_claimed_fees
            # self.xcp_profit += (self.xcp_profit - self.admin_claimed_fees) * (vp_new/vp_old - 1)
            # self.xcp_profit *= (vp_new / vp_old)
            self.xcp_profit += vp_new - vp_old
        else:
            self.xcp_profit *= vp_new / vp_old

        if rebalance_flag and vp_new > self.reserved_xcp_profit(self.xcp_profit) + self.offset:
            reb_vp = self.reserved_xcp_profit(self.xcp_profit)
            reb_xcp = reb_vp * self.total_supply
            self.xcp = reb_xcp
            self.vp = reb_vp
            self.rebalance_marks.append((self.step_count, self.vp))
        else:
            self.vp = vp_new
        return False

    def reserved_xcp_profit(self, xcp_profit) -> float:
        if "linear" in self.mode:
            return 1 + (xcp_profit - 1) / 2
        elif self.mode == "sqrt_admin":
            return np.sqrt(xcp_profit) - self.admin_claimed_fees
        elif "sqrt" in self.mode:
            return np.sqrt(xcp_profit)

    # -------------- simulation helpers ------------- #

    def _record(self) -> None:
        self.history["vp"].append(self.vp)
        self.history["xcp_profit"].append(self.xcp_profit)
        self.history["xcp_profit_a"].append(self.xcp_profit_a)
        self.history["reserved_xcp_profit"].append(self.reserved_xcp_profit(self.xcp_profit))
        self.history["admin_claimed_fees"].append(self.admin_claimed_fees)
        self.history["xcp"].append(self.xcp)

    def step(self, claim_interval, rebalance_interval) -> None:
        rebalance_flag = self.step_count % rebalance_interval == 0 and self.step_count > 0
        self.trade(rebalance_flag)
        if self.step_count % claim_interval == 0 and self.step_count > 0:
            self.claim_fees()
        self._record()
        self.step_count += 1

    # -------------- convenience run --------------- #

    def run(
        self, n_steps: int = 200, claim_interval: int = 50, rebalance_interval: int = 10
    ) -> None:
        for _ in range(n_steps):
            self.step(claim_interval, rebalance_interval)

In [ ]:
import matplotlib.pyplot as plt


claim_interval = 100
rebalance_interval = 1
xcp_growth = 0.001
n_steps = 500
offset = 0.01
admin_fee = 0.5
# Define models with different parameters
models = {
    "linear2": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="linear2",
        compound_mode="mul",
    ),
    "linear2, add": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="linear2",
        compound_mode="add",
    ),
    "linear1": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="linear1",
        compound_mode="mul",
    ),
    "sqrt": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="sqrt",
        compound_mode="mul",
    ),
    "sqrt, add": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="sqrt",
        compound_mode="add",
    ),
    "sqrt_lin": PoolModel(
        admin_fee=admin_fee, rebalance_offset=offset, xcp_growth=xcp_growth, mode="sqrt_lin"
    ),
    "sqrt_admin": PoolModel(
        admin_fee=admin_fee,
        rebalance_offset=offset,
        xcp_growth=xcp_growth,
        mode="sqrt_admin",
        compound_mode="mul",
    ),
}
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
]
# Run all models
for _, model in models.items():
    model.run(n_steps=n_steps, claim_interval=claim_interval, rebalance_interval=rebalance_interval)

# Plot all on the same axes
plt.figure(figsize=(10, 6))
for i, (key, model) in enumerate(models.items()):
    clr = colors[i]
    lw = 1
    plt.plot(model.history["vp"], label=f"vp, {key}", color=clr, linewidth=lw)
    plt.plot(
        model.history["xcp_profit"],
        label=f"xcp_profit, {key}",
        linestyle="-.",
        color=clr,
        linewidth=lw,
    )
    # xcpx = ((np.array(model.history['xcp_profit']) + np.array(model.history['xcp_profit_a']))/2 - 2*np.array(model.history['admin_claimed_fees']))
    # plt.plot(xcpx, label=f"xcpx, {key}", linestyle='-.', color=clr, linewidth=lw)

    # Optionally plot xcp_lp_share as well
    plt.plot(
        model.history["reserved_xcp_profit"],
        label=f"xcp_lp_share, {key}",
        linestyle=":",
        color=clr,
        linewidth=lw,
    )
    print(
        f"Model {i}, ({key}) has {len(model.rebalance_marks)} rebalances and {len(model.claim_marks)} claims"
    )

plt.title("Comparison of Pool Models")
plt.xlabel("step")
plt.ylabel("value")
plt.legend()
plt.tight_layout()
plt.show()